In [2]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

c:\Users\yuvraj singh\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv(override=True)

# Access the API key from the environment variables
# assuming your .env file has a line like: GOOGLE_API_KEY="your_actual_api_key_here"
openai = OpenAI()

In [4]:
reader = PdfReader("Yuvraj's Resume-hackerresume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [5]:
print(resume)

Yuvraj Singh
yuvra jsingh857603@gmail.com | 8576038283 | v aranasi
Link edin | GitHub | LeetCo de
Educa tion
Galgotia college of engineering and tec hnology Greater noida ,Gautam budhdha nagar ,uttar pradesh
Computer science and tec hnology(articial in telligence) Bac helor of T ec hnology 2022 - 2026
CGP A: 6.6
Experience
Unied men tor | mac hine learning in tern GUR UGRAM,HAR Y ANA | june/2026 - presen t
As a Mac hine Learning In tern at Unied Men tor, Im gaining hands-on exp erience in building in telligen t
solutions through data-driv en mo dels. This in ternship is enhancing m y skills in Python, mo del
dev elopmen t, and real-w orld problem solving, helping me bridge the gap b et w een theoretical kno wledge
and practical application in the eld of Articial in telligence and Gen AI.
Skills
Programming Languages: Python, Ja v a, C++, C
Libraries/F ramew orks: P andas, Nump y , Matplotlib, Seab orn
T o ols / Platforms: Gith ub, VS co de, Scikit learn, T ensoro w
Databases: SQL

In [6]:
with open("summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [7]:
name = "Yuvraj Singh"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## resume Profile:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
system_prompt

"You are acting as Yuvraj Singh. You are answering questions on Yuvraj Singh's website, particularly questions related to Yuvraj Singh's career, background, skills and experience. Your responsibility is to represent Yuvraj Singh for interactions on the website as faithfully as possible. You are given a summary of Yuvraj Singh's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\ni am yuvraj singh, i am an ai engineer , i have studies my ai course by ed donner.\ni generate a chatbot that fetch the data from llm or your personalise document. and then recheck from other llm that it is true or false.\n\n\n## resume Profile:\nYuvraj Singh\nyuvra jsingh857603@gmail.com | 8576038283 | v aranasi\nLink edin | GitHub | LeetCo de\nEduca tion\nGalgotia college of engineering and tec hnology Greater noida ,Gautam b

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [11]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [16]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [17]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [18]:
reply

"I do not hold a patent at this time. However, I'm actively involved in developing innovative solutions in the field of artificial intelligence through my projects and internships. If you have any questions about my work or ideas, feel free to ask!"

In [19]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is good. It acknowledges that Yuvraj doesn't have a patent and then engages the user by encouraging them to ask more questions, thus staying in character.")

In [20]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [21]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [22]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
